In [1]:
#웹 크롤링
import requests
from bs4 import BeautifulSoup

url = "https://www.hani.co.kr/arti/list.html?sec=news&subsec=politics"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")
titles = soup.find_all("h4", class_="article-title")
for title in titles:
    print(title.text.strip())


서울백병원 문 닫으면? 중구보건소 “도심 응급의료 공백 우려”
[속보]이규민 교육과정평가원장 “6월 모의평가 책임지고 사임”
당정 “수능에서 킬러 문항 배제… 자사고·외고·국제고는 존치”
평범한 우리도, 세상을 바꿀 수 있다
인간중심 간호 전문인재 양성…‘국시 100% 합격’
뜨겁다 뜨거워…폭염에도 삶은 계속되고, 방법은 다 있다
낮에 모인 ‘육퇴’ 동지들 “엄마가 행복해야 아이도 행복하다”
“오래된 물건 자랑하랬더니, 엄마아빠 쓰던 연필깎이 갖고 와”
‘오징어 게임 2’ 캐스팅 영상에 여성 0…넷플릭스에 물어보니
‘킬러 문항→사교육비→저출산’ 윤 대통령의 문제의식 배경
대안학교 가려면 지금부터 ‘발품’…2박3일 면접 보는 곳도
경기부양 욕심이 키운 ‘카드대란’ 연착륙에 성공하다
14살 스페이스X 신입사원 “채용 때 나이 편견 버리길 희망”
파업 유발시킨 ‘회사 불법’ 방치한 대법 판결…노란봉투법이 답이다
스위스, 법인세율 최저 11→15%로…국민투표 78.5% 찬성


In [35]:
# 다음 영화 예매 순위 정보
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://movie.daum.net/ranking/reservation'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

ol = soup.select_one('.list_movieranking')
li_list = ol.find_all('li')

movie = []

for li in li_list:
    rank = li.select_one('.rank_num').text
    name = li.select_one('.link_txt').text
    see = li.select_one('.ico_see').text
    grade = li.select_one('.txt_grade').text
    num = li.select_one('.txt_num').text[:-1]
    mvdate = li.select_one('.txt_info > .txt_num').text
    movie.append([rank, name, see, grade, num, mvdate])

df = pd.DataFrame(movie, columns=['순위', '영화명', '관람가', '평점', '예매율', '개봉일'])

df.to_csv('movie_info.csv', index=False, encoding='cp949')

df = pd.read_csv('movie_info.csv', encoding='cp949')
df.sort_values('평점', ascending=False)

df['개봉일'] = pd.to_datetime(df['개봉일'], format="%y.%m.%d")
df.head()

# rank_list = soup.select('.tit_item > a')
# rank_rate = soup.select('span.txt_append > span > span')

# for rank, title in enumerate(rank_list, 1):
#     print(f'{rank}위: {title.text} 예매율({rank_rate[rank-1].text})')


,순위,영화명,관람가,평점,예매율,개봉일
0,1,스파이더맨: 어크로스 더 유니버스,전체관람가,8.9,26.4,2023-06-21
1,2,귀공자,청소년관람불가,8.6,21.0,2023-06-21
2,3,엘리멘탈,전체관람가,8.8,16.0,2023-06-14
3,4,범죄도시3,15세이상관람가,6.7,13.1,2023-05-31
4,5,플래시,12세이상관람가,8.0,6.6,2023-06-14


## 과제
1. 웹크롤링
2. csv파일을 이용한 데이터 분석(전처리, 그래프)

두가지 합쳐서 해도 됨

In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://anime.onnada.com/'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

ol = soup.select_one('.layout-line')
ol
#li_list = ol.find_all('li')

SSLError: HTTPSConnectionPool(host='anime.onnada.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)')))